In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from IPython.display import display

In [36]:
# Leitura dos dados
df = pd.read_csv('../base_de_dados/application_test_student.csv')
df, df_teste = train_test_split(df, test_size=0.2)

df_teste.to_csv('../base_de_dados/application_train_test.csv', index=False)


df_explicativo = pd.read_csv('../base_de_dados/HomeCredit_columns_description.csv')

In [37]:
# Dropando coluna que define o sexo do cliente e coluna FONDKAPREMONT_MODE que possui dados nan demais.
df.drop(['CODE_GENDER', 'FONDKAPREMONT_MODE'], axis=1, inplace=True)

In [38]:
#Dropando colunas com 50% ou mais missing values
nulos = pd.DataFrame(df.isnull().sum(), columns=['total'])

muitos_nulos = nulos[nulos['total'] >= df.shape[0] * 0.5]

cols_muitos_nulos = muitos_nulos.index


df.drop(cols_muitos_nulos, axis=1, inplace=True)

In [39]:
#analise de discriminação - variaveis categoricas
for i in df.columns:
    
    if df[i].dtype=='object' or df[i].dtype=='int64':
        print(i)
        display(df.groupby(i).agg({'TARGET':['mean', 'count']}))

SK_ID_CURR


KeyError: "Column 'TARGET' does not exist!"

In [40]:
dropar = ['FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10',
          'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 
          'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21',
          'FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'NAME_CONTRACT_TYPE']

df.drop(columns=dropar, inplace=True, axis=1)

In [41]:
# Definindo Dataset com dados do tipo Object
df_obj = df.select_dtypes('O').copy()
df_obj

,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,EMERGENCYSTATE_MODE
2293,N,N,Unaccompanied,Working,Secondary / secondary special,Separated,House / apartment,Laborers,FRIDAY,Business Entity Type 3,NaN
42695,Y,Y,Unaccompanied,State servant,Secondary / secondary special,Married,House / apartment,Drivers,TUESDAY,Medicine,NaN
43004,Y,N,Unaccompanied,Working,Higher education,Married,With parents,NaN,TUESDAY,Business Entity Type 1,NaN
13379,N,Y,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,Drivers,MONDAY,Industry: type 3,No
42388,Y,Y,Family,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,FRIDAY,Business Entity Type 3,NaN
...,...,...,...,...,...,...,...,...,...,...,...
14459,Y,Y,NaN,Working,Secondary / secondary special,Married,House / apartment,Drivers,TUESDAY,Business Entity Type 3,NaN
55117,N,N,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Self-employed,NaN
7850,N,Y,Unaccompanied,Pensioner,Secondary / secondary special,Married,House / apartment,NaN,FRIDAY,XNA,NaN
51647,Y,Y,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,Security staff,THURSDAY,Business Entity Type 3,No


### Transformando colunas binárias categóricas em discretas

In [42]:
from sklearn.preprocessing import LabelEncoder

In [43]:
df_obj['NAME_TYPE_SUITE'].unique()

array(['Unaccompanied', 'Family', nan, 'Spouse, partner', 'Children',
       'Other_B', 'Other_A', 'Group of people'], dtype=object)

In [44]:
df_obj.isnull().sum()

FLAG_OWN_CAR                      0
FLAG_OWN_REALTY                   0
NAME_TYPE_SUITE                 194
NAME_INCOME_TYPE                  0
NAME_EDUCATION_TYPE               0
NAME_FAMILY_STATUS                0
NAME_HOUSING_TYPE                 0
OCCUPATION_TYPE               15438
WEEKDAY_APPR_PROCESS_START        0
ORGANIZATION_TYPE                 0
EMERGENCYSTATE_MODE           23261
dtype: int64

In [45]:
# Realizando a transformação em labels para cada coluna e salvando suas legendas

legendas = []


cols_cat = list(df_obj.columns)

for var in cols_cat:
    
    lbe = LabelEncoder()
    df_obj[var] = lbe.fit_transform(df[var].astype(str))
    
    legenda = pd.DataFrame(range(len(lbe.classes_)), index=lbe.classes_, columns=[var])
    legendas.append(legenda)
    

In [46]:
# DataFrame de legendas onde os -1 são valores que não existem na coluna
df_legendas = pd.concat(legendas)
df_legendas.fillna(-1, inplace=True)
df_legendas

,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE,EMERGENCYSTATE_MODE
N,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Y,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
N,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Y,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Children,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...
University,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,56.0,-1.0
XNA,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,57.0,-1.0
No,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
Yes,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0


In [47]:
#Exemplo de Uso
df_legendas[df_legendas['NAME_TYPE_SUITE'] != -1]['NAME_TYPE_SUITE']

Children           0.0
Family             1.0
Group of people    2.0
Other_A            3.0
Other_B            4.0
Spouse, partner    5.0
Unaccompanied      6.0
nan                7.0
Name: NAME_TYPE_SUITE, dtype: float64

### Preenchendo colunas para realizar a divisão em dummies

In [48]:
from sklearn.impute import SimpleImputer

In [49]:
si = SimpleImputer(strategy='most_frequent', missing_values=-1)
df_obj['NAME_TYPE_SUITE'] = si.fit_transform(df_obj[['NAME_TYPE_SUITE']])

In [50]:
df_obj['NAME_TYPE_SUITE'].unique()

array([6, 1, 7, 5, 0, 4, 3, 2])

In [51]:
from sklearn.impute import KNNImputer

In [52]:
knni = KNNImputer(missing_values=-1)
df_obj[cols_cat] = knni.fit_transform(df_obj)

In [53]:
# Voltando os valores para categóricos para separar em dummies
for col in cols_cat:
    legenda_col = df_legendas[df_legendas[col] != -1][col]
    
    dicionario = {}
    i = 0
    
    for legenda in legenda_col:
        dicionario[i] = legenda_col.index[i]
        i += 1
        
    df_obj[col] = df_obj[[col]].replace(dicionario)

### Transformando colunas em Dummies

In [54]:
from sklearn.preprocessing import OneHotEncoder

In [55]:
ohe = OneHotEncoder(drop='first', sparse=False)
ohe.fit(df_obj)

transform = ohe.transform(df_obj)

df_obj[list(range(transform.shape[1]))] = transform

In [56]:
# Trazendo os valores para o df original
df[df_obj.columns] = df_obj.values

In [57]:
df.drop(cols_cat, inplace=True, axis=1)

In [58]:
df

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,101,102,103,104,105,106,107,108,109,110
2293,452553,0,135000.0,299772.0,14580.0,247500.0,0.035792,-15804,-3509,-780.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
42695,377834,0,112500.0,1067418.0,37948.5,891000.0,0.005313,-16252,-658,-6846.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
43004,118793,0,81000.0,521280.0,27292.5,450000.0,0.035792,-11754,-1927,-5170.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13379,370233,0,247500.0,450000.0,28759.5,450000.0,0.035792,-9343,-825,-3916.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42388,241016,1,135000.0,589441.5,28485.0,441000.0,0.011703,-11189,-245,-7778.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14459,267076,0,202500.0,295677.0,7798.5,295677.0,0.026392,-21464,-989,-2685.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
55117,127328,0,135000.0,327024.0,15903.0,270000.0,0.028663,-10178,-1099,-409.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7850,408359,0,67500.0,135000.0,14323.5,135000.0,0.015221,-21694,365243,-11805.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
51647,307876,0,202500.0,260640.0,27499.5,225000.0,0.006207,-14579,-1063,-5999.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
from missingpy import MissForest
import time

In [61]:
x = df.drop(['TARGET', 'SK_ID_CURR'], axis=1)

mf = MissForest(max_depth=10, min_samples_split=16, max_features=0.7, max_iter=3, n_estimators=10)
mf.fit(x)

MissForest(max_depth=10, max_features=0.7, max_iter=3, min_samples_split=16,
           n_estimators=10)

In [62]:
inicio = time.time()
transform = mf.transform(x)
fim = time.time()

Iteration: 0
Iteration: 1
Iteration: 2


In [63]:
(inicio - fim)/60

-2.050980301698049

In [64]:
df[x.columns] = transform

In [65]:
df.isnull().sum().sum()

0

In [66]:
df.shape

(49202, 160)

In [67]:
#Removendo colunas que não serão utilizadas
df.drop(['FLOORSMAX_AVG', 'FLOORSMAX_MODE', 'FLOORSMAX_MEDI'], axis=1, inplace=True)

In [68]:
df.to_csv('../base_de_dados/tuning_application_test_student.csv', index=False)